# SETUP

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
import re
import pycountry
from countryinfo import CountryInfo
from currencies import Currency
import requests
import json
from bs4 import BeautifulSoup

# Airline and country

In [2]:
url = "https://www.pilotjobsnetwork.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

country_airlines = []

elements = soup.find_all("a", href=lambda href: href and "jobs" in href)
for element in elements :
    text = element.text.strip()
    country_airlines.append(text)

# country_airlines

In [3]:
airlines_by_country = []
for text in country_airlines:
    # Regex pattern to match the country and airline
    apattern = r'^[^-]*-[^-]*-\s*([^(]+)'
    cpattern = r'(\w+)\s\((\w{2})\)'
    

    country_match = re.search(cpattern, text)
    country = country_match.group(1)
    airline_match = re.search(apattern, text)
    airline = airline_match.group(1)

    # Append the airline and country to the list
    airlines_by_country.append((airline, country))

# Create the table
df = pd.DataFrame(airlines_by_country, columns=['Airline', 'Country'])
df['Country'] = df['Country'].replace('UK', 'United Kingdom')
# # Print the table
# df.head(10)

In [4]:
# Define countries standard name (ISO)
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return None
# Add a column to the dataframe with the ISO code of the country
df['ISO'] = df['Country'].apply(map_country_name)

In [5]:
df.head(10)

,Airline,Country,ISO
0,Eurowings GmbH,Germany,Germany
1,Greater Bay Airlines,Kong,Hong Kong
2,Eurowings Europe Ltd,Malta,Malta
3,Avion Express,Lithuania,Lithuania
4,Corendon Airlines,Turkey,Turkey
5,Hifly,Portugal,Portugal
6,Air Japan,Japan,Japan
7,Air Hamburg Luftverkehrsgesellschaft mbH,Germany,Germany
8,Aerowest,Germany,Germany
9,Austrian,Austria,Austria


In [6]:
df.to_csv("metadata_ctry_airlines.csv")

# Average gross income by country and Currency code (frozen)

In [7]:
url_3 = "https://www.worlddata.info/average-income.php"
df_3 = pd.read_html(url_3, attrs={'class': 'std100 hover', 'id': 'tabsort'})
df_3 = df_3[0]
df_3['Country'] = df_3['Country'].str.replace(' \*', '', regex=True)

In [8]:
# Define countries standard name (ISO)
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return None
# Add a column to the dataframe with the ISO code of the country
df_3['ISO'] = df_3['Country'].apply(map_country_name)

In [9]:
# Define a function to get the ISO code for each country
def get_iso_code(country_name):
    try:
        country = pycountry.countries.search_fuzzy(country_name)[0]
        return country.alpha_3
    except LookupError:
        return None
df_3['ISO_code'] = df_3['ISO'].apply(get_iso_code)

In [10]:
def get_currency(country):
    try:
        currency = CountryInfo(country).currencies()
        return currency
    except LookupError:
        return None
df_3['Currency'] = df_3['ISO'].apply(get_currency)

def first_item_to_string(arr):
    return str(arr[0])
# df_3['Currency'] = df_3['Currency'].apply(first_item_to_string)

In [12]:
df_3.columns = ['Rank', 'Country', 'yearly_income', 'monthly_income', 'ISO', 'ISO_code', 'Currency']

In [13]:
df_3["yearly_income"] = df_3["yearly_income"].str.replace(",", "").str.replace("$", "").astype(float)
df_3["monthly_income"] = df_3["monthly_income"].str.replace(",", "").str.replace("$", "").astype(float)
df_3["Currency"] = df_3["Currency"].str.replace("[',\[\]]", "")

/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_12680/68737043.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_3["yearly_income"] = df_3["yearly_income"].str.replace(",", "").str.replace("$", "").astype(float)
/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_12680/68737043.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_3["monthly_income"] = df_3["monthly_income"].str.replace(",", "").str.replace("$", "").astype(float)
/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_12680/68737043.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_3["Currency"] = df_3["Currency"].str.replace("[

In [14]:
df_3.to_csv("metadata_ctry_salary_2.csv")

# Currency Exchange rates

In [16]:
df_V1 = pd.read_csv("METADATA_V1.csv")
df_V1

,Country,year_income_gross,month_income_gross,ISO,Currency
0,Monaco,186080,15507,Monaco,EUR
1,Bermuda,122470,10206,Bermuda,BMD
2,Switzerland,90600,7550,Switzerland,CHF
3,Luxembourg,88190,7349,Luxembourg,EUR
4,Norway,83880,6990,Norway,NOK
...,...,...,...,...,...
64,Pakistan,1470,123,Pakistan,PKR
65,Nepal,1220,102,Nepal,NPR
66,Timor-Leste,1140,95,Timor-Leste,LCU
67,Ethiopia,940,78,Ethiopia,ETB


In [40]:
payload = {}
headers= {
  "apikey": "pPf6uwtTqHvmPDaWzHFzihcpKRAK4SC5"}
from_usd = "USD"
amount_convert = 1
def convert_usd(code_currency):
    try:
        response = requests.request("GET", f"https://api.apilayer.com/exchangerates_data/convert?to={code_currency}&from={from_usd}&amount={amount_convert}", headers=headers, data = payload)
        return json.loads(response.text)["info"]["rate"]
    except LookupError:
        return None

df_V1['XR'] = df_V1['Currency'].apply(convert_usd)

In [42]:
df_V2 = df_V1[["Currency", "XR"]]
df_V2 = df_V2.drop_duplicates()

,Currency,XR
0,EUR,0.937799
1,BMD,1.000000
2,CHF,0.932565
4,NOK,10.474465
6,USD,1.000000
7,DKK,6.978540
8,SGD,1.345925
9,ISK,140.579907
10,QAR,3.640992
11,SEK,10.503835


In [43]:
df_V2.to_csv("XR_list.csv")


# Index cost of living (frozen)

In [19]:
url_4 = "https://www.numbeo.com/cost-of-living/rankings_by_country.jsp"
df_4 = pd.read_html(url_4, attrs={'id': 't2'})
df_4 = df_4[0]

In [20]:
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return name
# Add a column to the dataframe with the ISO code of the country
df_4['ISO'] = df_4['Country'].apply(map_country_name)

df_4= df_4.replace("Hong Kong (China)", "Hong Kong")
df_4= df_4.replace("Macao (China)", "Macao")
df_4= df_4.replace("Taiwan (China)", "Taiwan")
df_4= df_4.replace("Kosovo (Disputed Territory)", "Kosovo")
df_4 = df_4[['ISO','Cost of Living Index']]
df_4 = df_4.dropna(how='any',axis=0) 

In [21]:
df_4.to_csv("cost_of_living_index.csv")

# Average salary income tax (frozen)

In [23]:
df_5 = pd.read_csv("salary_tax.csv")
# Define a function to map country names to their ISO codes
def map_country_name(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return name
# Add a column to the dataframe with the ISO code of the country
df_5['ISO'] = df_5['Country'].apply(map_country_name)
df_5["Last"].astype(float)
df_5.columns = ["Country", "Income_tax", "ISO"]

In [24]:
df_5.to_csv("income_tax_updated.csv")

# Inflation rate (frozen)

In [46]:
#Load inflation xls and add ISO countries
df_V3 = pd.read_csv("SQ_pred_database - 7 - Inflation.csv")
df_V3 

# Define a function to get the ISO name for each country
def get_iso_code(name):
    try:
        return pycountry.countries.search_fuzzy(name)[0].name
    except LookupError:
        return None
df_V3['ISO_name'] = df_V3['Country'].apply(get_iso_code)
df_V3

,ISO_code,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,ISO_name
0,ABW,Aruba,"2,08","4,32","0,63","-2,37","0,42","0,48","-0,89","-0,47","3,58","4,26","1,22","0,74","6,04",Aruba
1,AFG,Afghanistan,"2,18","11,8","6,44","7,39","4,67","-0,66","4,38","4,98","0,63","2,3","5,44","5,06",NaN,Afghanistan
2,AGO,Angola,"14,48","13,48","10,28","8,78","7,3","9,16","32,38","29,84","19,63","17,08","21,02","23,85","21,35",Angola
3,ALB,Albania,"3,57","3,41","2,03","1,93","1,62","1,91","1,29","1,99","2,03","1,41","1,62","2,04","6,73",Albania
4,ARE,United Arab Emirates,"0,88","0,88","0,66","1,09","2,34","4,07","1,62","1,97","3,06","-1,93","-2,08","0,18","5,22",United Arab Emirates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,Samoa,"-0,2","2,88","6,19","-0,21","-1,24","1,92","0,13","1,3","4,2","2,19","-1,57","-3,01","8,75",Samoa
189,YEM,"Yemen, Rep.","11,17","19,54","9,89","10,97","8,1","39,4",5,"30,4","27,6",10,"26,18","45,7","43,85",None
190,ZAF,South Africa,"4,06","5,02","5,72","5,78","6,14","4,51","6,59","5,27","4,5","4,13","3,22","4,61",7,South Africa
191,ZMB,Zambia,"8,5","6,44","6,59","6,99","7,82","10,11","17,86","6,58","7,49","9,8","15,73","22,02","10,99",Zambia


In [47]:
df_V3.to_csv("inlfation_rate_ISO.csv")

# Category

In [43]:
from tqdm.contrib.concurrent import thread_map
pd.set_option('display.max_colwidth', None)

#GET AIRLINES URLS
url = 'https://www.pilotjobsnetwork.com/'
max_urls = 5

def get_airlines_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href and 'jobs/' in href:
            airline_url = url + href
            yield airline_url
##GET INFO FROM PAGES
def get_airline_tables(airline_url):
    try:
        response = requests.get(airline_url)
        # Parse the HTML content of the response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find all text that contains the phrase "pilot jobs --->"
        matching_text = soup.find_all(text=lambda text: text and "pilot jobs --->" in text)
        # Regular expression to match text between parentheses
        for text in matching_text:
            paren_regex = r'\((.*?)\)'
            categories = []
            for text in matching_text:
                categories += re.findall(paren_regex, text)
            return [airline_url] + categories
    except NameError:
        return airline_url

# MAP IT 
list_category = list(thread_map(get_airline_tables, get_airlines_urls(url)))

#DF AS OUTPUT
df_category = pd.DataFrame([category for category in list_category if type(category) == list])

0it [00:00, ?it/s]/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_16502/4042108237.py:30: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  matching_text = soup.find_all(text=lambda text: text and "pilot jobs --->" in text)
501it [00:38, 13.13it/s]


,0,1
0,https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH,Major/National/Low Cost
1,https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines,Major/National/Low Cost
2,https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd,Major/National/Low Cost
3,https://www.pilotjobsnetwork.com/jobs/Avion_Express,Charter
4,https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines,Major/National/Low Cost
...,...,...
495,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,Fractional/Corporate
496,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,Fractional/Corporate
497,https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation,Regional
498,https://www.pilotjobsnetwork.com/jobs/Compass_Airlines,Major/National/Low Cost


In [55]:
# Define the regex pattern
regex_pattern = r'[^/]+(?=/?$)'

# Define a function to apply the regex pattern and replace underscores with spaces
def clean_url(url):
    match = re.search(regex_pattern, url)
    cleaned_url = match.group(0).replace('_', ' ')
    return cleaned_url
df_category[0] = df_category[0].apply(clean_url)
df_category.columns = ["Airline", "Category"]
df_category

,Airline,Category
0,Eurowings GmbH,Major/National/Low Cost
1,Greater Bay Airlines,Major/National/Low Cost
2,Eurowings Europe Ltd,Major/National/Low Cost
3,Avion Express,Charter
4,Corendon Airlines,Major/National/Low Cost
...,...,...
495,FlyinGroup,Fractional/Corporate
496,MS AVIATION,Fractional/Corporate
497,ESMA Aviation,Regional
498,Compass Airlines,Major/National/Low Cost


In [56]:
df_category.to_csv("category.csv")